In [6]:
# 0. bs4 모듈의 BeautifulSoup 클래스 사용
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from time import sleep

In [7]:
# 자주 사용하는 함수 정의
def bugs_row(tr):
    rank = tr.find('strong').text
    try:
        title = tr.find('p', {'class':'title'}).find('a').text
    except:
        title = tr.find('p', {'class':'title'}).find('span', '').text
    artist = tr.find('p', {'class':'artist'}).find('a').text
    try:
        album = tr.find('a', {'class':'album'}).text
    except:
        album = tr.find('span', {'class':'album'}).text
    img_url = tr.find('img').get('src').split('?')[0]
    row  = [rank, title, artist, album]
    return {'info':row, 'url':img_url}

In [8]:
# 1. 웹(html)문서 불러오기
#    1-2. 웹서버 접속해서 실시간으로 웹(html)문서 다운받기
url = 'https://music.bugs.co.kr/chart'

# response = headers + content(data,text)
# response.headers  : HTTP 헤더를 사전(Dict)으로 조회
# response.content  : HTTP 본문을 Data(Binary)형으로 조회, image, zip, hwp, ...
# response.text     : HTTP 본문을 Text(문자)형으로 조회, text/html
response = requests.get(url)
print('Response 본문의 데이터종류:', response.headers['Content-Type'])
print(response.text[:259])

Response 본문의 데이터종류: text/html;charset=UTF-8
<!DOCTYPE html>
<html lang="ko">
<head>
	<!-- bgsweb-www912 -->



	<meta charset="utf-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta name="keywords" content="노래추천, 음악추천, 플레이리스트, essential, 에센셜, 스트리밍, 스밍, 뮤직PD, 노래듣기, 음악다운로드, 페이코" />



In [9]:
# 2. 웹문서내용(str,문자열)을 분석 가능한 객체(bs4.BeautifulSoup) 생성
# 3. find, find_all 메소드등을 이용하여 특정한 <tag> 블럭 조회
text = response.text
html = BeautifulSoup(text, 'html.parser')
tr_list = html.find('table', {'class':'list'}).find('tbody').find_all('tr')

rows = []
for tr in tr_list:
    row = bugs_row(tr)['info']
    rows.append(row)
chart = pd.DataFrame(rows, columns = ['rank', 'title', 'artist', 'album'])
chart.set_index('rank', inplace = True)
chart

,title,artist,album
rank,,,
1,밤양갱,비비(BIBI),밤양갱
2,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’
3,나는 아픈 건 딱 질색이니까,(여자)아이들,2
4,EASY,LE SSERAFIM (르세라핌),EASY
5,홀씨,아이유(IU),The Winning
...,...,...,...
96,그대가 내 안에 박혔다(그내박),순순희(기태),그대가 내 안에 박혔다(그내박)
97,운이 좋았지,권진아,나의 모양
98,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles in December)’


In [ ]:
year = 2018
month = 1
for day in range(11, 13):
    date = f'{year:4}{month:0>2}{day:0>2}'
    path = f'bugs_{date}'
    if not os.path.isdir(path):
        os.mkdir(path)
    
    url = f'https://music.bugs.co.kr/chart/track/day/total?chartdate={date}'

    response = requests.get(url)
    text = response.text
    html = BeautifulSoup(text, 'html.parser')
    tr_list = html.find('table', {'class':'list'}).find('tbody').find_all('tr')

    rows = []
    for tr in tr_list:
        result = bugs_row(tr)
        row     = result['info']
        img_url = result['url']
        
        response2 = requests.get(img_url)
        content = response2.content
        filename = f'{path}/image_{date}_{row[0]}.jpeg'
        file = open(filename, 'wb')
        file.write(content)
        file.close()
        
        rows.append(row)
    chart = pd.DataFrame(rows, columns = ['rank', 'title', 'artist', 'album'])
    chart.set_index('rank', inplace = True)
    filename = f'{path}/bugs_chart_{date}.csv'
    chart.to_csv(filename, encoding = 'utf-8-sig')
    sleep(5)

In [91]:
date = '20180111'
url = f'https://music.bugs.co.kr/chart/track/day/total?chartdate={date}'

response = requests.get(url)
text = response.text
html = BeautifulSoup(text, 'html.parser')
tr_list = html.find('table', {'class':'list'}).find('tbody').find_all('tr')

In [122]:
rank37 = tr_list[36]
url = rank37.find('img').get('src').split('?')[0]
response = requests.get(url)
content = response.content

file = open('csv/그림.jpeg', 'wb')
file.write(content)
file.close()